In [ ]:
# gpu 가속 사용 했나 확인
!nvidia-smi

In [ ]:
# 파일 다운로드
!gdown https://drive.google.com/uc?id=1SpOi9QQgkBZbVPoWBWbnaeTZm-5eTDTY
!mkdir data
# 압축풀기 - content/data에 모든 압축 풀기
!unzip -qq /content/open.zip -d /content/data
!rm /content/open.zip
!mkdir /content/data/submission
!unzip -qq /content/data/sample_submission.zip -d /content/data/submission
!rm /content/data/sample_submission.zip


In [ ]:
import numpy as np
import random
import torch
import os
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from glob import glob
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,Normalizer

In [ ]:
class config:
    seed = 42
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    lr = 0.01
    epochs = 10
    batch_size = 640
    num_workers = 2
    train_5_folds = False

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
seed_everything(config.seed)
warnings.filterwarnings('ignore')

In [ ]:
all_input_list = sorted(glob('/content/data/train_input/*.csv'))
all_target_list = sorted(glob('/content/data/train_target/*.csv'))

In [ ]:
# train_validation split 어떻게?

# 전처리 어떻게?


In [ ]:
class MyDataset(Dataset):
    def __init__(self,input_list,target_list, scaler = None, val_mode = False):
        self.input_list = input_list
        self.target_list = target_list
        self.val_mode = val_mode
        self.data_list = []
        self.label_list = []
        self.scaler = scaler

        for input, target in tqdm(zip(self.input_list,self.target_list)):
            input_df = pd.read_csv(input).set_index('시간')
            target_df = pd.read_csv(target)
            
            # na 값 어떻게 할까?
            input_df.fillna(0,inplace=True)

            if self.scaler != None:
                input_df = scaler.fit_t

            input_length = int(len(input_df)/1440)
            target_length = int(len(target_df))

            for idx in range(target_length):
                time_series = input_df[1440*idx:1440*(idx+1)].values
                self.data_list.append(torch.Tensor(time_series))
            for label in target_df['rate']:
                self.label_list.append(label)

    def __getitem__(self,index):
        if self.val_mode:
            data = {'X':self.data_list[index],'y':self.target_list[index]}
        else:
            data = {'X':self.data_list[index]}
        return data

    def __len__(self):
        return len(self.data_list)

In [ ]:
input_df = pd.read_csv('/content/data/train_input/CASE_01.csv').set_index('시간')
input_df

In [ ]:
target_df = pd.read_csv('/content/data/train_target/CASE_01.csv')
target_df